In [1]:
import pdfplumber
import re
import requests
from io import BytesIO
from bs4 import BeautifulSoup
import pandas as pd
import time
from time import sleep
from tqdm import tqdm
import pymorphy3
from pymorphy3 import MorphAnalyzer
morph = MorphAnalyzer()

Для подсчета эмотивной лексики воспользуемся словарем проекта "Информационно-поисковая система «Эмоции и чувства в лексикографических параметрах: Словарь эмотивной лексики русского языка»", реализованного при поддержке Российского гуманитарного научного фонда. 
Система включает 37 pdf-файлов на странице, в каждом из которых лежат слова одного из классов эмоций, представленных в проекте. Пошагово получим слова из каждого класса.

In [2]:
#Спарсим страницу с систематизацией эмотивной лексики
url = "https://lexrus.ru/default.aspx?p=2876"
page = requests.get(url)
page.encoding = 'utf-8' 
soup = BeautifulSoup(page.text, 'html.parser')

In [3]:
#Получим категории эмоций и ссылки на соответствующие pdf-файлы
data = []
for link in soup.find_all('a'):
    if link.get('href') != None and 'inout/EMO' in link.get('href'):
        pdf_link = 'https://lexrus.ru/' + link.get('href')
        pdf_link = re.sub(r'\.\./', '', pdf_link)
        emotion_class = link.text
        data.append([emotion_class, pdf_link])

In [4]:
df = pd.DataFrame(data, columns=['Класс', 'Ссылка'])
df

,Класс,Ссылка
0,БЕСПОКОЙСТВО,https://lexrus.ru/inout/EMO/Bespokojstvo.pdf
1,ВДОХНОВЕНИЕ,https://lexrus.ru/inout/EMO/vdohnovenie.pdf
2,ВЕРА,https://lexrus.ru/inout/EMO/vera.pdf
3,ВЛЕЧЕНИЕ,https://lexrus.ru/inout/EMO/vlechenie.pdf
4,ВЫСОКОМЕРИЕ,https://lexrus.ru/inout/EMO/vysokomerie.pdf
5,ГОРЕ,https://lexrus.ru/inout/EMO/gore.pdf
6,ГРУСТЬ,https://lexrus.ru/inout/EMO/grust.pdf
7,ДОБРОТА,https://lexrus.ru/inout/EMO/dobrota.pdf
8,ДРУЖБА,https://lexrus.ru/inout/EMO/druzhba.pdf
9,ЖАЛОСТЬ,https://lexrus.ru/inout/EMO/zhalost.pdf


In [5]:
#Функция для парсинга pdf-файлов без скачивания
def parse_pdf(url):
    page = requests.get(url)
    with pdfplumber.open(BytesIO(page.content)) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    time.sleep(3)
    return text

In [6]:
df['Текст'] = df['Ссылка'].apply(parse_pdf)
df

,Класс,Ссылка,Текст
0,БЕСПОКОЙСТВО,https://lexrus.ru/inout/EMO/Bespokojstvo.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...
1,ВДОХНОВЕНИЕ,https://lexrus.ru/inout/EMO/vdohnovenie.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...
2,ВЕРА,https://lexrus.ru/inout/EMO/vera.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...
3,ВЛЕЧЕНИЕ,https://lexrus.ru/inout/EMO/vlechenie.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...
4,ВЫСОКОМЕРИЕ,https://lexrus.ru/inout/EMO/vysokomerie.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...
5,ГОРЕ,https://lexrus.ru/inout/EMO/gore.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...
6,ГРУСТЬ,https://lexrus.ru/inout/EMO/grust.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...
7,ДОБРОТА,https://lexrus.ru/inout/EMO/dobrota.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...
8,ДРУЖБА,https://lexrus.ru/inout/EMO/druzhba.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...
9,ЖАЛОСТЬ,https://lexrus.ru/inout/EMO/zhalost.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...


In [ ]:
#Функция для очистки текста из pdf-файла
def get_tokens(text):
    text = re.sub(r'\n|/', ' ', text)
    text = re.sub(r'(?<=[а-я])(?=[А-Я])', ' ', text) #разделим случайно склеившиеся слова
    text = re.sub('лексема|грамматическая|характеристика|дефиниция|иллюстрация|эмоциональное состояние|становление эмоционального состояния|эмоциональное воздействие|эмоциональное отношение|внешнее выражение эмоций|эмоциональная характеризация|эмоциональное качество', '', text, flags=re.IGNORECASE) #удалим категории из таблицы внутри pdf-файлов
    text = re.sub(r'[^а-яА-ЯёЁ\s-]', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = text.lower()
    text = text.strip()
    tokens = text.split()
    fixed_tokens = []
    #к некоторым словам случайно присоединяются числовые индексы как "з" или "г" в конце слова - исправим это так, чтобы случайно не удалить нужные буквы
    for token in tokens: 
        if morph.word_is_known(token) == False: #слова с подобными ошибками не находятся в словаре
            if token[-1] in 'зг': #если слово оканчивается на одну из этих букв, пробуем ее удалить и проверить, есть ли такое слово в словаре - если да, то берем эту версию
                new_token = token[:-1]
                if morph.word_is_known(new_token):
                    fixed_tokens.append(new_token)
        else:
            fixed_tokens.append(token)
    str_tokens = ' '.join(fixed_tokens) #токены соединяем обратно в строку, чтобы удобно было сохранить в csv
    return str_tokens

In [8]:
df['Токены'] = df['Текст'].apply(get_tokens)
df

,Класс,Ссылка,Текст,Токены
0,БЕСПОКОЙСТВО,https://lexrus.ru/inout/EMO/Bespokojstvo.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...,ажитация азарт аффект беспокоиться беспокойств...
1,ВДОХНОВЕНИЕ,https://lexrus.ru/inout/EMO/vdohnovenie.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...,вдохновение вдохновенный восторг наитие одухот...
2,ВЕРА,https://lexrus.ru/inout/EMO/vera.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...,вера вериться самонадеянный самообман самоувер...
3,ВЛЕЧЕНИЕ,https://lexrus.ru/inout/EMO/vlechenie.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...,дерзание дерзновение домогательство маньячеств...
4,ВЫСОКОМЕРИЕ,https://lexrus.ru/inout/EMO/vysokomerie.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...,барский высокомерный заносчивый погордиться пр...
5,ГОРЕ,https://lexrus.ru/inout/EMO/gore.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...,беда бессчастный боль выстрадать горе горевать...
6,ГРУСТЬ,https://lexrus.ru/inout/EMO/grust.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...,безнадежность безотрадность безысходность грус...
7,ДОБРОТА,https://lexrus.ru/inout/EMO/dobrota.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...,альтруизм беззлобие безропотный бескорыстие бе...
8,ДРУЖБА,https://lexrus.ru/inout/EMO/druzhba.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...,благорасположенный дружелюбный либеральный мир...
9,ЖАЛОСТЬ,https://lexrus.ru/inout/EMO/zhalost.pdf,Лексема Грамматическая Дефиниция Иллюстрация\n...,жалость жаль каяться покаяться милосердие раск...


In [ ]:
#Сохраним результат в csv
df.to_csv('emot_lex.csv')